In [1]:
%%capture
import subprocess, importlib
if importlib.util.find_spec('import-ipynb') is None:
    !pip install import-ipynb
    
import import_ipynb
from DME_Load import *

In [2]:
#Cooling Water
for s in [S[x] for x in [5, 6, 7]]:
    set_stream(
        s,
        Y=pure(H2O_l),
        n=1.0e-10 * ureg.mol / ureg.hr,
        T=design.cooling.temperature,
        P=design.cooling.pressure
    )
    
#Raw Water
for s in [S[x] for x in [11, 12]]:
    set_stream(
        s,
        Y=pure(H2O_l),
        n=1.0e-10 * ureg.mol / ureg.hr,
        T=design.environment.ambient,
        P=design.raw_water.pressure,
    )

#Chilled Water
for s in [S[x] for x in []]:
    set_stream(
        s,
        Y=pure(H2O_l),
        n=1.0e-10 * ureg.mol / ureg.hr,
        T=design.chilled_water.minimum_temperature,
        P=design.chilled_water.pressure
    )

In [ ]:
@suppress_warning(pint.UnitStrippedWarning)
def solve_conveyor(conveyor: Plant) -> None:
    c_in = get_stream(conveyor.inlets.S)
    P_drop = getattr(conveyor, "pressure drop", Q_(0, ureg.kPa))    
    c_out = get_stream(conveyor.outlets.S)
    
    if not hasattr(c_in, "K") and not hasattr(c_in, "CV") and not hasattr(c_in, "T") and not hasattr(c_in, "A"):
        set_proximate_stream(c_out, K=None, T=None, CV=None, A=None)
        # print(f"1")
        
    ##################
    ##HIDDEN PORTION##
    ##################
    
    system_review(
        conveyor.name, in_streams=[conveyor.inlets.S], out_streams=[conveyor.outlets.S]
    )

In [ ]:
@suppress_warning(pint.UnitStrippedWarning)
def solve_bin(bin: Plant) -> None:
    b_in = get_stream(bin.inlets.S)
    b_out = get_stream(bin.outlets.S)
     
    if not hasattr(b_in, "K") and not hasattr(b_in, "CV") and not hasattr(b_in, "T") and not hasattr(b_in, "A"):
        set_proximate_stream(b_out, K=None, T=None, CV=None, A=None)
        #print(f"1")
        
    ##################
    ##HIDDEN PORTION##
    ##################
    
    system_review(
        bin.name, in_streams=[bin.inlets.S], out_streams=[bin.outlets.S]
    )


In [ ]:
design.feed.ultimate_analysis_dry
pdt = ['C', 'H', 'O', 'N', 'Si(Ash)', 'Cl', 'S']

for x, y in zip(pdt, design.feed.ultimate_analysis_dry): #x is element in pdt, y is element in design feed 
    print(f"{x}: {y}")

=======Ultimate Analysis (Dry)=======
C: 45.40 %w/w
H: 5.70 %w/w
O: 46.20 %w/w
N: 0.80 %w/w
Si(Ash): 1.80 %w/w
Cl: 0.00 %w/w
S: 0.10 %w/w


In [6]:
with design.feed as fd:
    fd.A = np.matmul(
        qarray([1 - fd.moisture, fd.moisture]),
        qarray([fd.ultimate_analysis_dry.to_base_units(), A_MOISTURE()]),
    ).to(ureg.pbw)

    set_proximate_stream(
        S[101],
        L_dry=fd.proximate_composition_dry,
        x_moisture=fd.moisture,
        m=fd.nominal_feed_rate,
        T=fd.temperature,
        rho=fd.bulk_density,
        CV_dry=fd.calorific_value_dry,
        A=fd.A,
    )

with S[101] as f:
    f.CV_L = qarray(L_0(), unit=ureg.MJ / ureg.kg)
    f.CV_L[FixedCarbon] = qarray(-HHV_(Y_FIXEDCARBON()))
    f.CV_L[Volatiles] = (f.CV - np.dot(f.L, f.CV_L)) / f.L[Volatiles]

f = None
fd = None

In [7]:
solve_conveyor(units["VV-201"])
#TODO: conveyor sizing


========================================Feed seal VV-201=========================================
---------------------------------------------
                        101          102     
                        Feed     Feed airlock
                      material    discharge  
---------------------------------------------
Mass           kg/h          340          340
Flow           m³/h            -            -
Pressure        kPa         -0.0         -0.0
Temperature      °C         25.0         25.0
---------------------------------------------
Heat           MJ/h          3.1          3.1
HHV            MJ/h      -4482.9      -4482.9
---------------------------------------------
 
mass input                               340.00 kg/h
mass output                              340.00 kg/h
net mass flow                            0.00 kg/h                 

heat flows                               3.05 MJ/h 3.05 MJ/h
HHV flows                                4482.90 MJ/h 4482.90 MJ/

In [8]:
solve_bin(units["BN-202"])
#TODO: Need to add bin volume / surge capacity.


=======================================Feed Hopper BN-202========================================
---------------------------------------------
                        102          201     
                    Feed airlock     SGP     
                     discharge    discharge  
---------------------------------------------
Mass           kg/h          340          340
Flow           m³/h            -            -
Pressure        kPa         -0.0         -0.0
Temperature      °C         25.0         25.0
---------------------------------------------
Heat           MJ/h          3.1          3.1
HHV            MJ/h      -4482.9      -4482.9
---------------------------------------------
 
mass input                               340.00 kg/h
mass output                              340.00 kg/h
net mass flow                            0.00 kg/h                 

heat flows                               3.05 MJ/h 3.05 MJ/h
HHV flows                                4482.90 MJ/h 4482.90 MJ/

In [9]:
solve_conveyor(units["CV-203"])


========================================Feed screw CV-203========================================
---------------------------------------------
                        201          202     
                        SGP       Pyrolysis  
                     discharge    kiln feed  
---------------------------------------------
Mass           kg/h          340          340
Flow           m³/h            -            -
Pressure        kPa         -0.0         -0.0
Temperature      °C         25.0         25.0
---------------------------------------------
Heat           MJ/h          3.1          3.1
HHV            MJ/h      -4482.9      -4482.9
---------------------------------------------
 
mass input                               340.00 kg/h
mass output                              340.00 kg/h
net mass flow                            0.00 kg/h                 

heat flows                               3.05 MJ/h 3.05 MJ/h
HHV flows                                4482.90 MJ/h 4482.90 MJ/

In [ ]:
def solids_pyrolysis(K_feed, final_volatile_content, mobilised_carbon):
    """Using the feed proximate mass vector and the final volatile content (assumed) returns the proximate mass vector of char after pyrolysis"""
    v = final_volatile_content
    K = K_feed * (L_FIXEDCARBON() * (1 - mobilised_carbon) + L_ASH())
    return (K + v / (1 - v) * sum(K) * L_VOLATILES()) / ureg.wf

def pyrolyse(B, T, b_tar):
    """Convert an ultimate analysis (B) at temperature T into a component analysis

    :param B: Element vector
    :type B: pint.Quantity containing a ndarry of dimension N_ELEMENTS
    :param T: Temperature of pyrolysis
    :type T: pint.Quantity [temperature]
    :param tar: fraction which is tar
    :type tar: float

    :return: composition of pyrolysis gas
    :rtype: pint.Quantity containing a ndarry of dimension N_COMPONENTS
    """
    ##################
    ##HIDDEN PORTION##
    ##################

In [11]:
with S[units["KN-204"].inlets.S] as f:
    f.M_A = f.A.to_reduced_units() * f.m
    print(f.M_A)

[115.77 24.05 193.30 2.04 4.59 0.00 0.26] kg/h


In [ ]:
def proximate_to_elementz(L, A_vols):
    A_vols = A_vols / (100 * ureg.pbw)    
    return np.matmul(L, qarray([A_FIXEDCARBON(), A_ASH(), A_vols, A_MOISTURE()]))

In [ ]:
with S[units["KN-204"].outlets.S] as s, S[units["KN-204"].inlets.S] as f:
    ##################
    ##HIDDEN PORTION##
    ##################

    print(s)

==================================== 203 : Kiln char discharge =====================================
 tag                                  : 203 : Kiln char discharge
 name                                 : Kiln char discharge
 fluid                                : char
 m                                    : 37.31 kg/h
 T                                    : 600 °C
 rho                                  : 195 kg/m³
 P                                    : 101.3 kPa
 v                                    : 0.1914 m³/h
--------------------------------- Proximate Vectors / Coordinates ----------------------------------
                      K          L
Symbol             kg/h       %w/w
──────────────────────────────────
FixedCarbon       31.98      85.70
Ash                4.59      12.30
Volatiles          0.75       2.00
──────────────────────────────────
Total             37.31     100.00

--------------------------------- Elemental Vectors / Coordinates ------------------------------

## Raw Syngas (without dust)

In [ ]:
with S[units["KN-204"].outlets.G] as sg, S[units["KN-204"].outlets.S] as s, S[units["KN-204"].inlets.S] as f:
    sg.m = f.m - s.m

    ##################
    ##HIDDEN PORTION##
    ##################

    #Assuming the discharge char temperature and syngas temperature are the same

    sg = None
    s = None
    f = None
    print(S[204])

    set_stream(S[205],
               T = S[204].T,
               P = S[204].P,
               M = S[204].M
              )

[115.77 24.05 193.30 2.04 4.59 0.00 0.26] kg/h
[32.26 0.05 0.41 0.01 4.59 0.00 0.00] kg/h


/opt/conda/lib/python3.11/site-packages/numpy/matrixlib/defmatrix.py:146: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  arr = N.array(data, dtype=dtype, copy=copy)


====================================== 204 : Syngas from Kiln ======================================
 tag                                  : 204 : Syngas from Kiln
 name                                 : Syngas from Kiln
 fluid                                : raw syngas
 m                                    : 302.7 kg/h
 T                                    : 600 °C
 P                                    : 101.3 kPa
 rho                                  : 0.2859 kg/m³
 v                                    : 1,059 m³/h
--------------------------------- Proximate Vectors / Coordinates ----------------------------------
                      K          L
Symbol             kg/h       %w/w
──────────────────────────────────
FixedCarbon        1.68       0.56
Volatiles        216.00      71.36
Moisture          85.00      28.08
──────────────────────────────────
Total            302.69     100.00

--------------------------------- Elemental Vectors / Coordinates ----------------------------

## Factoring for Dust

In [ ]:
units["KN-204"].dust = Plant("dust entrained")
with units["KN-204"].dust as dst, S[units["KN-204"].outlets.S] as s:
    dst.rho = s.rho * 0.7
    ##################
    ##HIDDEN PORTION##
    ##################
    print(dst)

dst = None

========================================== dust entrained ==========================================
 tag                                  : dust entrained
 rho                                  : 136.5 kg/m³
 fraction                             : 2.5 %w/w
 m                                    : 0.9328 kg/h
--------------------------------- Proximate Vectors / Coordinates ----------------------------------
                      L          K
Symbol             %w/w       kg/h
──────────────────────────────────
FixedCarbon       87.45       0.82
Ash               12.55       0.12
──────────────────────────────────
Total            100.00       0.93

--------------------------------- Elemental Vectors / Coordinates ----------------------------------
                    M_A        B_N
Symbol             kg/h      mol/h
──────────────────────────────────
C                  0.82      67.92
Si                 0.12       4.17
──────────────────────────────────
Total              0.93      72.0

## Final Products

On the basis that the char size proportioning and hindered settling model will be tackled in the future, the current setup is for a final concentration of dust in gas. Separation is then made on that basis. NOTE: Volatiles in char is made part of the initial setup of S203.

In [ ]:
with units["KN-204"].dust as dst, S[units["KN-204"].outlets.S] as ch, S[units["KN-204"].outlets.G] as sg, S[units["KN-204"].inlets.S] as fd:
    ch.K -= dst.K
    ##################
    ##HIDDEN PORTION##
    ##################
    assert np.all(np.abs((sg.M_A + ch.M_A - fd.M_A).m) <= 1e-10)

print(sg)
print(ch)

====================================== 204 : Syngas from Kiln ======================================
 tag                                  : 204 : Syngas from Kiln
 name                                 : Syngas from Kiln
 fluid                                : raw syngas
 m                                    : 303.6 kg/h
 T                                    : 600 °C
 P                                    : 101.3 kPa
 rho                                  : 0.2859 kg/m³
 v                                    : 1,059 m³/h
 HHV                                  : -3,482 MJ/h
 Q                                    : 575 MJ/h
--------------------------------- Proximate Vectors / Coordinates ----------------------------------
                      K          L
Symbol             kg/h       %w/w
──────────────────────────────────
FixedCarbon        2.50       0.56
Ash                0.12       0.00
Volatiles        216.00      71.36
Moisture          85.00      28.08
─────────────────────────────

==================================== 203 : Kiln char discharge =====================================
 tag                                  : 203 : Kiln char discharge
 name                                 : Kiln char discharge
 fluid                                : char
 m                                    : 36.38 kg/h
 T                                    : 600 °C
 rho                                  : 195 kg/m³
 P                                    : 101.3 kPa
 v                                    : 0.1914 m³/h
 HHV                                  : -1,033 MJ/h
 Q                                    : 21.1 MJ/h
--------------------------------- Proximate Vectors / Coordinates ----------------------------------
                      K          L
Symbol             kg/h       %w/w
──────────────────────────────────
FixedCarbon       31.16      85.70
Ash                4.47      12.30
Volatiles          0.75       2.00
──────────────────────────────────
Total             36.38     10

## Discharge Cooler

In [ ]:
def solve_cooler(cooler: Plant) -> None:
    c_in = get_stream(cooler.inlets.C)
    ##################
    ##HIDDEN PORTION##
    ##################
    set_stream(
        c_out, M=c_in.M, Q=c_in.Q + Q_cooling.Q, P=c_in.P - P_drop, allow_gas=False
    )
    
    system_review(
        cooler.name,
        in_streams=[cooler.inlets.C, cooler.inlets.Q],
        out_streams=[cooler.outlets.C],
    )

In [18]:
set_stream(
    get_stream(units["KN-204.C"].inlets.C),
    X=pure(H2O),
    m=design.cooling.kiln_discharge_rate,
    T=design.environment.dew_point.to(ureg.K) + design.cooling.dew_point_deltaT,
    P=design.cooling.pressure,
)

#Need to set stream KN204 inlets Q of the heat loss. What is the surface area?
set_heat_stream(
    get_stream(units["KN-204.C"].inlets.Q),
    Q=units["KN-204.C"].cooling_area * units["KN-204.C"].cooling_rate,
)

solve_cooler(units["KN-204.C"])


=======================================Kiln cooling jacket=======================================
----------------------------------------------------------
                         5           216          903     
                      Cooling      Cooling    Kiln cooler 
                    water inlet  water return heat losses 
                     to KN-204   from KN-204              
----------------------------------------------------------
Mass           kg/h          660          660            0
Flow           m³/h        890.3          0.0          0.0
Pressure        kPa        200.0        175.0          0.0
Temperature      °C         23.0         27.8          0.0
----------------------------------------------------------
Heat           MJ/h       1609.4       1615.9          6.4
HHV            MJ/h          0.0          0.0          0.0
----------------------------------------------------------
 
mass input                               660.00 kg/h
mass output         

/opt/conda/lib/python3.11/site-packages/pint/facets/plain/quantity.py:1165: RuntimeWarning: divide by zero encountered in scalar divide
  return self.__class__(other_magnitude / self._magnitude, 1 / self._units)


## Heat of Reaction

In [19]:
with S[units["KN-204"].outlets.G] as sg, S[units["KN-204"].inlets.S] as fd, S[units["KN-204"].outlets.S] as ch:
    fd.CV_L_dry = design.feed.calorific_value_dry
    sg.Y_dry = frac(sg.Y*(1-pure(H2O))).to(ureg.pbm)
    sg.CV = -HHV_(sg.Y_dry.to_reduced_units()).to(ureg.MJ / ureg.kg)

    units["KN-204"].Q_pyrolysis = -(sg.HHV + ch.HHV - fd.HHV).to(ureg.MJ/ureg.hr)
    print(units["KN-204"].Q_pyrolysis)


31.31 MJ/h


In [20]:
#This bit is to update the volatiles HHV value in all the streams to the calculated rather than given values. There appears to be missing attributes in here - need to examine.

attr = ["m", "T", "Q", "HHV"]
attr_m = ["Q", "HHV"]

for n in [102, 201, 202, 204, 203]: #there is no 103 
    print(n,"".join([f"{f'{getattr(S[n], a, None)}':>16}" for a in attr]),
         "".join([f"{f'{getattr(S[n], a, 0.0)/S[n].m}':>16}" for a in attr_m]),
         "".join([f"{f'{a}':>16}" for a in getattr(S[n], "K", L_0()*Q_(1, ureg.kg/ureg.hr))]))


102      340.00 kg/h        25.00 °C       3.05 MJ/h   -4482.90 MJ/h       0.01 MJ/kg    -13.18 MJ/kg       33.66 kg/h       4.59 kg/h     216.75 kg/h      85.00 kg/h
201      340.00 kg/h        25.00 °C       3.05 MJ/h   -4482.90 MJ/h       0.01 MJ/kg    -13.18 MJ/kg       33.66 kg/h       4.59 kg/h     216.75 kg/h      85.00 kg/h
202      340.00 kg/h        25.00 °C       3.05 MJ/h   -4482.90 MJ/h       0.01 MJ/kg    -13.18 MJ/kg       33.66 kg/h       4.59 kg/h     216.75 kg/h      85.00 kg/h
204      303.62 kg/h       600.00 °C     575.01 MJ/h   -3481.65 MJ/h       1.89 MJ/kg    -11.47 MJ/kg        2.50 kg/h       0.12 kg/h     216.00 kg/h      85.00 kg/h
203       36.38 kg/h       600.00 °C      21.10 MJ/h   -1032.56 MJ/h       0.58 MJ/kg    -28.38 MJ/kg       31.16 kg/h       4.47 kg/h       0.75 kg/h       0.00 kg/h


## Heat Required

In [ ]:
with units["KN-204"] as kn:
    len = kn.dimensions.base_frame.length
    wid = kn.dimensions.base_frame.width
    hei = (kn.dimensions.base_frame.height + kn.dimensions.cover.height)
    ##################
    ##HIDDEN PORTION##
    ##################
    
    print(kn.net_specific_heat_required)
kn = None

3.32 MJ/kg


# Char Cooling Subsytem

In [22]:
S[313].T = design.cooling.temperature

char_cooling = Plant("Char Cooling Subsystem")

with S[units["CV-306"].inlets.S] as i, S[units["CV-308"].outlets.S] as o:
    T, P = (i.T.to(ureg.K) - Q_(250, ureg.K)).to(ureg.degC), i.P
    T_l = s.T = design.environment.ambient

    set_proximate_stream(S[901], K = i.K, T = T)
    Q_loss = -(S[901].Q - i.Q)
    m_w = (Q_loss/(2257*ureg.kJ/ureg.kg)).to_reduced_units()
    set_stream(S[11], X = pure(H2O_l), m = m_w, T = T_l, P = design.raw_water.pressure)

    o.T = design.product.maximum_temperature
    o.m = i.m
    o.M_A = i.M_A
    o.K = i.K
    o.v = i.v
    o.rho = i.rho
    o.Q = heat_solids(o.K, o.T).to(ureg.MJ / ureg.hr)
    o.HHV = -np.dot(o.K, S[101].CV_L)

    o = None
    i = None

with char_cooling as cc:
    cc.in_solids = S[units["CV-306"].inlets.S]
    cc.out_solids = S[units["CV-308"].outlets.S]
    cc.out_solids.P = cc.in_solids.P
    cc.Q_cooling = S[units["CV-308"].outlets.S].Q - S[901].Q

    cc.in_cw = S[313]
    cc.out_cw = S[310]

    with cc.in_cw as i, cc.out_cw as o:
        i.T = S[313].T
        o.T = Q_((i.T.to(ureg.K) + 15 * ureg.K).to(ureg.degC)) ##asume dT is 15 K/degC
        i.L = o.L = L_MOISTURE()
        i.m = o.m = -(
            cc.Q_cooling / Q_(4.184, ureg.kJ / ureg.kg / ureg.K) / (o.T - i.T)
        ).to_reduced_units()
        i.v = o.v = i.m / design.cooling.density
        i.M = pure(H2O_l) * i.m
        o.M = pure(H2O_l) * o.m
        #o.P = i.P 
        
        i = None
        o = None

        cc.U = Q_(5, ureg.W / ureg.m**2 / ureg.K)  #Probably should put this into yaml file.
        cc.ΔTh = cc.in_solids.T - cc.out_cw.T
        cc.ΔTc = cc.out_solids.T - cc.in_cw.T
        cc.lmtd = (cc.ΔTh - cc.ΔTc) / np.log(cc.ΔTh / cc.ΔTc)
        cc.A_min = (cc.Q_cooling / cc.U / cc.lmtd).to_base_units()

    cc = None

/opt/conda/lib/python3.11/site-packages/pint/facets/numpy/numpy_func.py:307: RuntimeWarning: invalid value encountered in log
  result_magnitude = func(*stripped_args, **stripped_kwargs)


In [ ]:
##################
##HIDDEN PORTION##
##################

char_cooling.A = units["CV-306"].A + units["CV-307"].A + units["CV-308"].A
print(units["CV-306"].A)

1994283.02 mm²


In [24]:
char_cooling.in_cw.v   #THIS SEEMS ODDLY NEGATIVE - TODO: NEED TO CHECK

0.1713188217697813 <Unit('meter ** 3 / hour')>

## FIBC Filling

Clause 9.1 provides that bags will be provided with a loading capacity of a minimum of $1m^3$.

In [ ]:
bag_filling = Plant("product filling")
bag_filling.bin = units["BN-311"]
bag_filling.valve = units["VV-312"]

##################
##HIDDEN PORTION##
##################

with bag_filling.bin as b, bag_filling.valve as v, S[305] as i, S[307] as o:
    # Estimated size of RV
    v.d_est = Q_(200, ureg.mm)
    v.h_allowance = Q_(800, ureg.mm)

    ##################
    ##HIDDEN PORTION##
    ##################
    b.h_cone = (b.d - v.d_est) / 2 * np.tan(b.alpha)
    b.h_rev_cone = b.d * np.tan(b.rill)
    b.V_min = design.product.product_holding * i.v
    b.V_gross = (np.pi / 4 * b.d**2 * (b.h + b.h_cone / 3)).to(ureg.m**3)
    b.V = (np.pi / 4 * b.d**2 * ((b.h - b.h_rev_cone) + b.h_cone / 3 + b.h_rev_cone / 3)).to(
        ureg.m**3
    )
    b.h_t = (b.h + b.h_cone + Q_(1000, ureg.mm)).to(ureg.m)

    b.material = "Carbon Steel"
    b.lining = "low friction spray"

    v.fill_factor = 0.95  # fill factor for valve

#print(bag_filling)
b = None
v = None
f = None
i = None
o = None


=====================================Char Seal Valve VV-309======================================
---------------------------------------------
                        303          304     
                      Cooling      Char to   
                     Conveyor 3     Bucket   
                     discharge     Elevator  
---------------------------------------------
Mass           kg/h           36           36
Flow           m³/h          0.2          0.2
Pressure        kPa          0.0         -0.0
Temperature      °C         45.0         45.0
---------------------------------------------
Heat           MJ/h          0.9          0.9
HHV            MJ/h      -1032.6      -1032.6
---------------------------------------------
 
mass input                               36.38 kg/h
mass output                              36.38 kg/h
net mass flow                            0.00 kg/h                 

heat flows                               0.91 MJ/h 0.91 MJ/h
HHV flows            

**Polisher Separation** is the amount of contaminants/components removed by the exit of the polisher.

In [27]:
design.gcu.polisher_separation = qarray(
    pure(C1) * design.gcu.removal_efficiencies.dust
    + pure(Si1) * design.gcu.removal_efficiencies.dust
    + pure(H2O_l) * 1.0
    + pure(CH4O) * design.gcu.removal_efficiencies.tar2
    + pure(C7H8) * design.gcu.removal_efficiencies.tar1
    + pure(C7H8O) * design.gcu.removal_efficiencies.tar2
    + pure(C10H8) * design.gcu.removal_efficiencies.tar3
    + pure(C16H10) * design.gcu.removal_efficiencies.tar4
    + pure(NH3) * design.gcu.removal_efficiencies.amines
    + pure(H2S) * design.gcu.removal_efficiencies.sulphides
    + pure(HCl) * design.gcu.removal_efficiencies.chlorides,
    unit=ureg.pbm,
).to(ureg.pbm)

In [28]:
design.gcu.scrubber_separation = qarray(
    pure(C1) * design.gcu.removal_efficiencies.dust
    + pure(Si1) * design.gcu.removal_efficiencies.dust
    + pure(H2O_l) * 1.0
    + pure(CH4O) * design.gcu.removal_efficiencies.tar2 * 0.98
    + pure(C7H8) * design.gcu.removal_efficiencies.tar1 * 0.98
    + pure(C7H8O) * design.gcu.removal_efficiencies.tar2 * 0.99
    + pure(C10H8) * design.gcu.removal_efficiencies.tar3 * 0.99
    + pure(C16H10) * design.gcu.removal_efficiencies.tar4 * 0.99
    + pure(NH3) * design.gcu.removal_efficiencies.amines * 0.90
    + pure(H2S) * design.gcu.removal_efficiencies.sulphides * 0.99
    + pure(HCl) * design.gcu.removal_efficiencies.chlorides * 0.99,
    unit=ureg.pbm,
).to(ureg.pbm)

In [29]:
design.gcu.conditioner_separation = qarray(
    pure(C1) * design.gcu.removal_efficiencies.dust
    + pure(Si1) * design.gcu.removal_efficiencies.dust
    + pure(H2O_l) * 1.0
    + pure(CH4O) * design.gcu.removal_efficiencies.tar2 * 0.1
    + pure(C7H8) * design.gcu.removal_efficiencies.tar1 * 0.02
    + pure(C7H8O) * design.gcu.removal_efficiencies.tar2 * 0.10
    + pure(C10H8) * design.gcu.removal_efficiencies.tar3 * 0.30
    + pure(C16H10) * design.gcu.removal_efficiencies.tar4 * 0.50
    + pure(NH3) * design.gcu.removal_efficiencies.amines * 0.20
    + pure(H2S) * design.gcu.removal_efficiencies.sulphides * 0.50
    + pure(HCl) * design.gcu.removal_efficiencies.chlorides * 0.50,
    unit=ureg.pbm,
).to(ureg.pbm)

In [ ]:
##################
##HIDDEN PORTION##
##################
S[421].P = S[205].P

In [ ]:
##################
##HIDDEN PORTION##
##################
S[421].T = S[205].T

In [ ]:
set_stream(
    ##################
    ##HIDDEN PORTION##
    ##################
)
S[406].relative_humidity_operational = water_relative_humidity(
    S[406].N, T=T_REF, P=design.gcu.syngas_header_pressure + P_REF
).to(ureg.pbm)

print(S[406])

======================================== 406 : Clean syngas ========================================
 tag                                  : 406 : Clean syngas
 name                                 : Clean syngas
 fluid                                : syngas
 P                                    : 111.3 kPa
 T                                    : 35 °C
 n                                    : 9.153 kmol/h
 m                                    : 188.5 kg/h
 rho                                  : 0.8144 kg/m³
 v                                    : 231.4 m³/h
 Q                                    : 10.18 MJ/h
 H_f                                  : -1,126 MJ/h
 HHV                                  : -2,847 MJ/h
 LHV                                  : -2,596 MJ/h
 relative_humidity_operational        : 85.27 %mol/mol
-------------------------------- Composition Vectors / Coordinates ---------------------------------
                      N          M          Y          X
Symbol          

In [33]:
set_stream(
    S[405],
    m=units["CL-403"].scrubber.gas_mass_rate,
    Y=S[406].Y,
)

In [34]:
set_stream(S[407], 
           M=S[405].M-S[406].M, T=S[406].T, P=S[406].P)


print(S[407])

#assert all(S[407].M.m>=-1)  # positive mass balance
stream_table(S[405], S[406], S[407], long_form=True, label=" Syngas Blower ")

================================= 407 : Syngas recycle to scubber ==================================
 tag                                  : 407 : Syngas recycle to scubber
 name                                 : Syngas recycle to scubber
 fluid                                : syngas
 P                                    : 111.3 kPa
 T                                    : 35 °C
 n                                    : 185.1 kmol/h
 m                                    : 3,812 kg/h
 rho                                  : 0.8144 kg/m³
 v                                    : 4,680 m³/h
 Q                                    : 206 MJ/h
 H_f                                  : -2.277e+04 MJ/h
 HHV                                  : -5.757e+04 MJ/h
 LHV                                  : -5.25e+04 MJ/h
-------------------------------- Composition Vectors / Coordinates ---------------------------------
                      N          M          Y          X
Symbol            mol/h       kg/h  

### Dehumidifier, Reheater

**Reheater outlet** being the same mass flow as the **dehumidifer outlet** and increased in temperature.

In [35]:
set_stream(
    S[404],
    M=S[405].M
)

**Polisher outlet** balance based on having all components 

In [ ]:
set_stream(
    S[402],
    ##################
    ##HIDDEN PORTION##
    ##################
    )
    * (1 - pure(H2O_l)),
)
assert all(S[402].M.m>=-1)  # positive mass balance

**Dehumidifier outlet** before gas/liquid separation and **liquid** return from dehumidifier

In [37]:
set_stream(
    S[419],
    N=condense_water(S[402].N - S[404].N),
)
set_stream(
    S[403],
    N=S[404].N + S[419].N,
)
assert all(S[419].M.m>=0)  # positive mass balance

In [38]:
set_stream(S[407], 
           M=S[405].M-S[406].M, T=S[406].T, P=S[406].P)

assert all(S[407].M.m>=-1)  # positive mass balance

set_stream(S[427],
           M=S[405].M, T=S[406].T, P=S[406].P)

stream_table(S[405], S[406], S[407], S[427], long_form=True, label=" Syngas Blower ")

========================================= Syngas Blower =========================================
-----------------------------------------------------------------------
                        405          406          407          427     
                     Syngas to   Clean syngas    Syngas     Syngas ex  
                       blower                  recycle to     blower   
                                                scubber                
-----------------------------------------------------------------------
Mass           kg/h         4000          188         3812         4000
Flow           m³/h       4672.4        231.4       4680.0       4911.5
Pressure        kPa         -8.0         10.0         10.0         10.0
Temperature      °C         20.0         35.0         35.0         35.0
-----------------------------------------------------------------------
Heat           MJ/h        121.7         10.2        206.0        216.1
HHV            MJ/h     -60421.9      

In [39]:
set_stream(S[9],
            Y=pure(H2O_l),
            n=1.0e-10 * ureg.mol / ureg.hr,
            T=design.chilled_water.minimum_temperature,
            P=design.chilled_water.pressure + P_REF)

set_stream(S[420],
           M=S[9].M,
           T=S[9].T.to(ureg.K) + Q_(4, ureg.K),
           P=S[9].P - Q_(100, ureg.kPa))

design.gcu.chilled_water_flow =(S[402].Q-S[403].Q)/(S[420].Q/S[420].m-S[9].Q/S[9].m)

set_stream(S[9],
           m= design.gcu.chilled_water_flow,
           X=S[9].X)
set_stream(S[420],
           m= design.gcu.chilled_water_flow,
           X=S[420].X)
stream_table(S[402], S[403], S[404], S[419], S[9], S[420], long_form=True, label=" Dehumidifier and Reheater Section ")

=============================== Dehumidifier and Reheater Section ===============================
-------------------------------------------------------------------------------------------------
                        402          403          404          419           9           420     
                     Syngas to      Cooled       Cooled     liquor ex     Chilled      chilled   
                    dehumidifer  dehumidified dehumidified  dehumifier    water to     water ex  
                                 syngas post  syngas post               dehumidifier  dehumifier 
                                 condensation condensation                 HX-407                
                                      IN          OUT                                            
-------------------------------------------------------------------------------------------------
Mass           kg/h         4079         4079         4000           79        19180        19180
Flow           m³/h 

In [ ]:
set_stream(
    S[450],
    ##################
    ##HIDDEN PORTION##
    ##################
        S[450].P)*(1-pure(H2O_l))
)

In [41]:
#stream_table(*[S[x] for x in [10, 11, 12]], long_form=True)

set_stream(   
    S[12],
    X=pure(H2O_l),
    m=design.gcu.makeup_water,
)

In [42]:
set_stream(
    S[451],
    N=condense_water(S[12].N+S[450].N-S[402].N),
    allow_gas=False
)

assert all(S[451].M.m>=-1e-10)  # positive mass balance

In [43]:
set_stream(
    S[418],
    M=S[451].X*units["CL-403"].polisher.recirculation_rate + S[12].M,
    allow_gas=False,
)
set_stream(
    S[417],
    M = S[451].X * units["CL-403"].polisher.recirculation_rate,
    Q = S[418].Q - S[12].Q,
    allow_gas=False,
)
set_stream(
    S[416],
    M=S[451].X*units["CL-403"].polisher.recirculation_rate,
    Q=S[417].Q + S[405].Q - S[404].Q,
    allow_gas=False,
)

In [44]:
set_stream(
    S[451],
    M=S[451].M,
    T=S[416].T
)
stream_table(*[S[x] for x in [450, 402]], long_form=True,label=" Polisher Balance - Gas")
stream_table(*[S[x] for x in [416, 417, 12, 418, 451]], long_form=True,label=" Polisher Balance - Liquid") #fixed streams

===================================== Polisher Balance - Gas=====================================
---------------------------------------------
                        450          402     
                    vapour from   Syngas to  
                    scrubber to  dehumidifer 
                      polisher               
---------------------------------------------
Mass           kg/h         4095         4079
Flow           m³/h       4962.5       4907.6
Pressure        kPa         -4.2         -5.0
Temperature      °C         30.0         28.0
---------------------------------------------
Heat           MJ/h        416.4        364.2
HHV            MJ/h     -60430.8     -60422.3
---------------------------------------------
 
=================================== Polisher Balance - Liquid====================================
------------------------------------------------------------------------------------
                        416          417           12          418       

In [ ]:
set_stream(
    S[13],
    ##################
    ##HIDDEN PORTION##
    ##################
    allow_gas=False
    )

set_stream(
    S[14], 
    ##################
    ##HIDDEN PORTION##
    ##################
    allow_gas=False
    )

In [46]:
(units["CL-401"].water_exit_temperature.to(ureg.K) - units["FL-110"].temperature_loss_from_conditioner.to(ureg.K)).to(ureg.degC)

45.0 <Unit('degree_Celsius')>

In [ ]:
set_stream(
    S[150],
    M = condense_water(    
    ##################
    ##HIDDEN PORTION##
    ##################
)
assert all(S[150].M.m>=-1)  # positive mass balance

print(S[150])

==================================== 150 : Total waste from gcu ====================================
 tag                                  : 150 : Total waste from gcu
 name                                 : Total waste from gcu
 fluid                                : liquor
 n                                    : 15.24 kmol/h
 m                                    : 287.4 kg/h
 T                                    : 45 °C
 rho                                  : 1,051 kg/m³
 v                                    : 0.2735 m³/h
 Q                                    : 23.03 MJ/h
 P                                    : 101.3 kPa
 H_f                                  : -4,300 MJ/h
 HHV                                  : -607.4 MJ/h
 LHV                                  : 81.11 MJ/h
-------------------------------- Composition Vectors / Coordinates ---------------------------------
                      N          M          Y          X
Symbol            mol/h       kg/h   %mol/mol       %w/w

In [48]:
design.gcu.solids_vector = composition(
    [C1, Si1, C16H10, C10H8, C7H8O, CH4O, H2S], [1.0, 1.0, 1.0, 1.0, 0.20, 0.10, 0.50]
)  # Mass basis

In [49]:
set_stream(
    S[152],
    M = S[150].M * design.gcu.solids_vector
)
set_stream(
    S[153],
    M = S[150].M - S[152].M
)    

In [50]:
units["FL-110"].efficiency

98 <Unit('parts_per_hundred')>

In [51]:
set_stream(
    S[111],
    M = (S[152].M  + (1- units["FL-110"].cake_solids)/units["FL-110"].cake_solids*S[153].X*S[152].m)*units["FL-110"].efficiency,
    T=S[150].T,
    P = S[150].P,
    allow_gas = False
)
with S[111] as ck:
    ck.K = np.sum(
        qarray(pure(C1), pure(Si1), (1 - pure(C1) - pure(Si1) - pure(H2O_l)), pure(H2O_l))
        * ck.M,
        axis=-1,
    )
    ck.L = frac(ck.K).to(ureg.pbw)
    ck.M_A = as_elements(ck.N) * ATOMIC_WEIGHT
    ck.m = np.sum(ck.K)

In [52]:
set_stream(
    S[110],
    M = S[111].M - S[150].M, #150 - 111
    T= S[150].T,
    P = S[150].P,
    allow_gas = False
)
stream_table(*[S[x] for x in [150, 110, 111]], long_form=True,label=" GCU Waste Material ")
print(output_string(S[110].X))

====================================== GCU Waste Material =======================================
----------------------------------------------------------
                        150          110          111     
                    Total waste  Filter waste Filter cake 
                      from gcu      water      ex filter  
----------------------------------------------------------
Mass           kg/h          287         -248           39
Flow           m³/h          0.3         -0.2          0.0
Pressure        kPa          0.0          0.0          0.0
Temperature      °C         45.0         45.0         45.0
----------------------------------------------------------
Heat           MJ/h         23.0        -20.4          2.6
HHV            MJ/h       -607.4        200.2       -407.1
----------------------------------------------------------
 


                       
Symbol             %w/w
───────────────────────
CH4O               0.51
C7H8               0.67
C7H8O              0.85
C10H8              0.03
C16H10             0.05
NH3                0.29
H2S                0.05
H2O_l             97.56
───────────────────────
Total            100.00



In [53]:
set_stream(
    S[7],
    X = pure(H2O_l),
    m = units["CL-403"].scrubber.cooler_flow_rate,
)

In [54]:
design.gcu.gas_dew_point = dew_point(S[205].N).to(ureg.degC)
print(design.gcu.gas_dew_point)

74.79 °C


In [ ]:
#water liquor side then amount required --> meb balance
def conditioner_above_dew_point():
    T, P = (S[401].T.to(ureg.K) - 5 * ureg.K).to(ureg.degC), S[401].P
    T_l = S[413].T
    set_stream(
        S[457],
        N=S[205].N * (1 - design.gcu.conditioner_separation.to_reduced_units()),
        T=T,
        P=P
    )
    ##################
    ##HIDDEN PORTION##
    ##################
    
    ##################
    ##HIDDEN PORTION##
    ##################
    Q_evap = heat(N_evap, T) - heat(condense_water(N_evap), T_l)      # <<<<<<<<<<<<<<<<
    if Q_evap > Q_cooling:
        warnings.warn("Evaporative cooling exceeds cooling duty")
    N_liq = pure(H2O_l) * ((Q_cooling - Q_evap)/heat(pure(H2O_l), T, T_l)).to(ureg.mol/ureg.hr)
    set_stream(
        S[413],
        N = condense_water(N_evap+N_liq),
        P = units["CL-401.S"].spray_pressure,
    )
    set_stream(
        S[401],
        N = N_evap + S[457].N)
    set_stream(
        S[408],
        N = N_liq + S[205].N * design.gcu.conditioner_separation.to_reduced_units(),
        T = T,
        P = P,
        allow_gas = False
    )

In [ ]:
#water condense from gas side then amount of liquor required --> meb balance
def conditioner_below_dew_point():
    T, P = (S[401].T.to(ureg.K) - 5 * ureg.K).to(ureg.degC), S[401].P
    T_l = S[413].T
    set_stream(
        S[457],
        N=S[205].N * (1 - design.gcu.conditioner_separation.to_reduced_units()),
        T=T,
        P=P
    )
    Q_cooling = S[205].Q - S[457].Q
    ##################
    ##HIDDEN PORTION##
    ##################
    
    ##################
    ##HIDDEN PORTION##
    ##################                                          # <<<<<<<<<<<<<<<<

    Q_evap = (n_evap*(heat(pure(H2O),T)-heat(pure(H2O_l),T))).to(ureg.MJ/ureg.hr)                   # <<<<<<<<<<<<<<<<
    
    if Q_evap > Q_cooling:
        warnings.warn("Evaporative cooling exceeds cooling duty")
    N_liq = pure(H2O_l) * ((Q_cooling - Q_evap)/heat(pure(H2O_l), T, T_l)).to(ureg.mol/ureg.hr)
    set_stream(
        S[413],
        N = N_liq,                                                                                  # <<<<<<<<<<<<<<<<
        P = units["CL-401.S"].spray_pressure,
    )
    set_stream(
        S[401],
        N = n_evap*pure(H2O) + S[457].N)                                                            # <<<<<<<<<<<<<<<<
    set_stream(
        S[408],
        N = N_liq + S[205].N * design.gcu.conditioner_separation.to_reduced_units() - n_evap*pure(H2O_l),  # <<<<<<<<<<<<<<<<
        T = T,
        P = P,
        allow_gas = False
    )

In [57]:
if units["CL-401"].exit_temperature > design.gcu.gas_dew_point:
    conditioner_above_dew_point()
else:
    conditioner_below_dew_point()

system_review("Conditioner", [205, 413], [401, 408])


===========================================Conditioner===========================================
-----------------------------------------------------------------------
                        205          401          408          413     
                     Bypass gas   Syngas to    Liquor to     Water to  
                    valve outlet   scrubber   conditioner  conditioner 
                                                 filter       spray    
-----------------------------------------------------------------------
Mass           kg/h          303          238         3154         3090
Flow           m³/h       1058.7        317.0          3.2          3.1
Pressure        kPa          0.0         -1.0         -1.0        198.7
Temperature      °C        600.0         65.0         60.0         25.0
-----------------------------------------------------------------------
Heat           MJ/h        574.3        115.6        460.9          0.0
HHV            MJ/h      -3454.9     

In [58]:
set_stream(S[458],
           N=S[408].N,
           P=S[408].P,
           T=S[111].T, 
          allow_gas=False)

set_stream(
    S[108],
    N=S[408].N - S[111].N,
    T=S[111].T,
    P=S[111].P
, 
          allow_gas=False)

system_review("Filter", [458,], [108, 111])

set_stream(
    S[109],
    N=S[108].N-S[110].N,
    T=S[108].T,
    P=S[108].P, 
          allow_gas=False)
system_review("Waste water", [108], [109, 110])


=============================================Filter==============================================
----------------------------------------------------------
                        108          111          458     
                      Filtrate   Filter cake   S408 with  
                    from filter   ex filter     expected  
                                              heat losses 
----------------------------------------------------------
Mass           kg/h         3115           39         3154
Flow           m³/h          3.2          0.0          3.2
Pressure        kPa          0.0          0.0         -1.0
Temperature      °C         45.0         45.0         45.0
----------------------------------------------------------
Heat           MJ/h        260.8          2.6        263.4
HHV            MJ/h        231.3       -407.1       -175.8
----------------------------------------------------------
 
mass input                               3154.45 kg/h
mass output        

In [ ]:
def solve_scrubber():
    with units["CL-403"].scrubber as sc:
    ##################
    ##HIDDEN PORTION##
    ##################
        print(sc)


solve_scrubber()


In [ ]:
set_stream(S[412],
           Y=S[413].Y,
           m=Q_(9000, ureg.kg/ureg.hr),
           P=S[413].P,
           T=S[413].T)

set_stream(S[411],
           M=S[413].M + S[412].M,
           T=S[412].T,
           P=S[412].P)

In [ ]:
set_stream(S[410],
           M=S[411].M-S[13].M-S[14].M,
           P=S[411].P,
           Q=S[412].Q+S[109].Q+S[401].Q-S[450].Q+S[407].Q+S[451].Q)
S[410].T = S[410].T.to(ureg.degC)

print(S[410].Q - S[411].Q)

set_stream(
    S[415],
    M = S[7].M,
    T = (((S[410].Q - S[411].Q)/(S[7].m * 4.184 * ureg.kJ/ureg.kg/ureg.K)) + S[7].T.to(ureg.K)).to(ureg.degC)
)
stream_table(*[S[x] for x in [7, 415]], long_form=True,label=" GCU Cooling ")

201.76 MJ/h
========================================== GCU Cooling ==========================================
---------------------------------------------
                         7           415     
                      Cooling      cooling   
                      water to    water from 
                       HX-405      scrubber  
                                  exchanger  
---------------------------------------------
Mass           kg/h        25000        25000
Flow           m³/h         25.0         25.0
Pressure        kPa        200.0          0.0
Temperature      °C         15.0         16.9
---------------------------------------------
Heat           MJ/h      -1047.1       -844.9
HHV            MJ/h          0.0          0.0
---------------------------------------------
 


In [ ]:
system_review("Scrubber Balance", [401, 109, 451, 407, 412], [450, 410])


========================================Scrubber Balance=========================================
-------------------------------------------------------------------------------------------------
                        109          401          407          410          412          450     
                      Scrubber    Syngas to      Syngas     liquor ex     Water to   vapour from 
                    water top up   scrubber    recycle to    scrubber     scrubber   scrubber to 
                                                scubber        pump        spray       polisher  
-------------------------------------------------------------------------------------------------
Mass           kg/h         3363          238         3812        12088         9000         4095
Flow           m³/h          3.4        317.0       4680.0         12.1          9.0       4962.5
Pressure        kPa          0.0         -1.0         10.0        198.7        198.7         -4.2
Temperature      °C

In [ ]:
def set_burner_streams(
    burner: Plant,
    m_input: pint.Quantity = None,
    Q_input: pint.Quantity = None,
    T_flame: float = None,
    y_O2: float = None,
    excess_air: float = None,
    T_flue: pint.Quantity = None,
    efficiency: float = 1.0,
):
    """
    ##################
    ##HIDDEN PORTION##
    ##################

    """
    with burner as bu:
        for attribute, default in [
        ##################
        ##HIDDEN PORTION##
        ##################

        with bu.fuel as f, bu.air as a, bu.flue_gas as g:
        ##################
        ##HIDDEN PORTION##
        ##################
            bu.oxygen_wet_basis = g.Y[O2]

In [ ]:
design.fuel.gas_pressure_g

In [ ]:
gc = design.fuel.gas_composition
                                          
gascomp = composition([CH4, C2H6, C3H8, C4H10, CO2, N2, O2],
                       [gc.CH4*100, gc.C2H6*100, gc.C3H8*100, gc.C4H10*100, gc.CO2*100, gc.N2*100, gc.O2*100], 
                      ureg.pbm)

#############################################################
with S[918] as fg:
    set_stream(
        fg,
        Y=gascomp,
        n=1.0e-10 * ureg.mol / ureg.hr,
        T=design.environment.ambient,
        P=(Q_(design.fuel.gas_pressure_g) + P_REF),
    )
    
    ##################
    ##HIDDEN PORTION##
    ##################

stream_table(S[918], S[406], long_form=True)

Relative density: 0.58
nu: 41.57 mol/m³
gross_wobbe_index: 48.79 MJ/m³
---------------------------------------------
                        918          406     
                     Fuel usage  Clean syngas
---------------------------------------------
Mass           kg/h            0          188
Flow           m³/h          0.0        231.4
Pressure        kPa        140.0         10.0
Temperature      °C         13.0         35.0
---------------------------------------------
Heat           MJ/h         -0.0         10.2
HHV            MJ/h         -0.0      -2847.2
---------------------------------------------
 


In [ ]:
units["KN-204.B"] = Plant("Kiln Burner")

with units["KN-204.B"] as bu:
    bu.operating = Plant("Kiln Burner System on syngas")
    bu.startup = Plant("Kiln Burner System on natural gas")
    bu.FTcomb = Plant("Kiln Burner System on FT excess gas")

    with bu.operating as op: #syngas
        op.fuel = S[601] #syngas to kiln 
        op.fuel.Y = S[406].Y #clean syngas; .Y = composition  

    ##################
    ##HIDDEN PORTION##
    ##################


        set_burner_streams(
            burner=op,
            Q_input=units["KN-204"].net_heat_required,
            y_O2=design.pyrolysis.dry_oxygen,
            T_flue=design.pyrolysis.exhaust_temperature,
        )
    
    with bu.startup as st: #LPG
    ##################
    ##HIDDEN PORTION##
    ##################

        
    with bu.FTcomb as FT: #Fischer Tropsch Excess Gas
    ##################
    ##HIDDEN PORTION##
    ##################

    print(bu.operating)
    print(bu.startup)
    print(bu.FTcomb)


print(units["KN-204"].net_heat_required)
print(S[2])

1896.99 °C
1898.71 °C
1831.12 °C


=================================== Kiln Burner System on syngas ===================================
 tag                                  : Kiln Burner System on syngas
 fuel
    tag                               : 601 : Syngas to kiln
    name                              : Syngas to kiln
    fluid                             : syngas
    T                                 : 25 °C
    P                                 : 101.3 kPa
    n                                 : 5.269 kmol/h
    m                                 : 108.5 kg/h
    rho                               : 0.8417 kg/m³
    v                                 : 128.9 m³/h
    Q                                 : 4.153 MJ/h
    H_f                               : -648.4 MJ/h
    HHV                               : -1,639 MJ/h
    LHV                               : -1,495 MJ/h
 air
    tag                               : 208 : Kiln combustion air ex Recuperator
    name                              : Kiln combustion air ex R

================================ Kiln Burner System on natural gas =================================
 tag                                  : Kiln Burner System on natural gas
 fuel
    tag                               : 2 : Fuel to kiln
    name                              : Fuel to kiln
    fluid                             : fuel
    T                                 : 25 °C
    P                                 : 101.3 kPa
    n                                 : 1.857 kmol/h
    m                                 : 30.85 kg/h
    rho                               : 0.6792 kg/m³
    v                                 : 45.42 m³/h
    Q                                 : 0 MJ/h
    H_f                               : -141.2 MJ/h
    HHV                               : -1,654 MJ/h
    LHV                               : -1,491 MJ/h
 air
    tag                               : operating air
    T                                 : 200 °C
    P                                 : 101.3 kPa
 

=============================== Kiln Burner System on FT excess gas ================================
 tag                                  : Kiln Burner System on FT excess gas
 fuel
    tag                               : 612 : FT Excess Gas
    name                              : FT Excess Gas
    fluid                             : syngas
    T                                 : 25 °C
    P                                 : 101.3 kPa
    n                                 : 3.131 kmol/h
    m                                 : 96.51 kg/h
    rho                               : 1.26 kg/m³
    v                                 : 76.61 m³/h
    Q                                 : 0.5374 MJ/h
    H_f                               : -601.9 MJ/h
    HHV                               : -1,646 MJ/h
    LHV                               : -1,513 MJ/h
 air
    tag                               : operating air
    T                                 : 200 °C
    P                                 : 

========================================= 2 : Fuel to kiln =========================================
 tag                                  : 2 : Fuel to kiln
 name                                 : Fuel to kiln
 fluid                                : fuel
 T                                    : 25 °C
 P                                    : 101.3 kPa
 n                                    : 1.857 kmol/h
 m                                    : 30.85 kg/h
 rho                                  : 0.6792 kg/m³
 v                                    : 45.42 m³/h
 Q                                    : 0 MJ/h
 H_f                                  : -141.2 MJ/h
 HHV                                  : -1,654 MJ/h
 LHV                                  : -1,491 MJ/h
-------------------------------- Composition Vectors / Coordinates ---------------------------------
                      Y          N          M          X
Symbol         %mol/mol      mol/h       kg/h       %w/w
──────────────────────

In [ ]:
with units["HX-206"] as eq, S[eq.inlets.H] as hi, S[eq.outlets.H] as ho, S[eq.inlets.C] as ci, S[eq.outlets.C] as co:       
    ##################
    ##HIDDEN PORTION##
    ##################
    eq.delta_Q = co.Q - ci.Q
    
    eq.hot_out_temp = TsolveH(hi.N, hi.Q - eq.delta_Q - heat_of_vaporisation(hi.N).to(ureg.MJ / ureg.hr)).to(ureg.degC)
    ##################
    ##HIDDEN PORTION##
    ##################
    print(eq)
        
eq = None
hi = None
ho = None
ci = None
co = None

print(S[208])
print(S[209])
print(S[210])
print(S[211])

=============================== HX-206 : Combustion air recuperator ================================
 tag                                  : HX-206 : Combustion air recuperator
 name                                 : Combustion air recuperator
 type                                 : heat exchanger
 inlets
    tag                               : inlets
    H                                 : 210
    C                                 : 209
 outlets
    tag                               : outlets
    H                                 : 211
    C                                 : 208
 hot_side
    tag                               : hot_side
    pressure_drop                     : 1 kPa
 cold_side
    tag                               : cold_side
    inlet_temperature                 : 25 °C
    pressure_drop                     : 1 kPa
 delta_Q                              : 102.6 MJ/h
 hot_out_temp                         : 419.6 °C
 td1                                  : 350 Δ°C
 td2   

============================= 208 : Kiln combustion air ex Recuperator =============================
 tag                                  : 208 : Kiln combustion air ex Recuperator
 name                                 : Kiln combustion air ex Recuperator
 fluid                                : air
 T                                    : 200 °C
 P                                    : 101.3 kPa
 n                                    : 18.7 kmol/h
 m                                    : 539.4 kg/h
 rho                                  : 0.7431 kg/m³
 v                                    : 725.9 m³/h
 Q                                    : 96.03 MJ/h
 H_f                                  : 0 MJ/h
 HHV                                  : -8.131e-13 MJ/h
 LHV                                  : -8.538e-13 MJ/h
-------------------------------- Composition Vectors / Coordinates ---------------------------------
                      Y          N          M          X
Symbol         %mol/mol    

================================ 209 : Kiln combustion air fan exit ================================
 tag                                  : 209 : Kiln combustion air fan exit
 name                                 : Kiln combustion air fan exit
 fluid                                : air
 n                                    : 18.7 kmol/h
 m                                    : 539.4 kg/h
 T                                    : 13 °C
 rho                                  : 1.229 kg/m³
 v                                    : 439 m³/h
 Q                                    : -6.538 MJ/h
 P                                    : 101.3 kPa
 H_f                                  : 0 MJ/h
 HHV                                  : -8.131e-13 MJ/h
 LHV                                  : -8.538e-13 MJ/h
-------------------------------- Composition Vectors / Coordinates ---------------------------------
                      N          M          Y          X
Symbol            mol/h       kg/h   %mol/

======================================== 210 : Kiln exhaust ========================================
 tag                                  : 210 : Kiln exhaust
 name                                 : Kiln exhaust
 fluid                                : flue gas
 P                                    : 101.3 kPa
 T                                    : 550 °C
 n                                    : 22.3 kmol/h
 m                                    : 647.9 kg/h
 rho                                  : 0.4301 kg/m³
 v                                    : 1,507 m³/h
 Q                                    : 540.3 MJ/h
 H_f                                  : -2,143 MJ/h
 HHV                                  : -1.205e-12 MJ/h
 LHV                                  : -1.165e-12 MJ/h
-------------------------------- Composition Vectors / Coordinates ---------------------------------
                      Y          N          M          X      Y_dry
Symbol         %mol/mol      mol/h       kg/h     

================================ 211 : Kiln exhaust ex recuperator =================================
 tag                                  : 211 : Kiln exhaust ex recuperator
 name                                 : Kiln exhaust ex recuperator
 fluid                                : flue gas
 n                                    : 22.3 kmol/h
 m                                    : 647.9 kg/h
 T                                    : 419.6 °C
 rho                                  : 0.5111 kg/m³
 v                                    : 1,268 m³/h
 Q                                    : 437.8 MJ/h
 P                                    : 99.83 kPa
 H_f                                  : -2,143 MJ/h
 HHV                                  : -1.205e-12 MJ/h
 LHV                                  : -1.165e-12 MJ/h
-------------------------------- Composition Vectors / Coordinates ---------------------------------
                      N          M          Y          X
Symbol            mol/h      

In [ ]:
set_stream(S[421],
           M = S[205].M,
           T = S[205].T,
           P = S[205].P
          )

In [ ]:
with design.engines as e, S[406] as csg, S[602] as eg, S[603] as sacg, Plant("Temp") as t:
    e.xa = 0.15
    e.ng_afr = air_to_fuel_ratio(Y_NG()) * ureg.mf * (1 + e.xa)
    ##################
    ##HIDDEN PORTION##
    ##################
        
    e.gas = eg
    print(e)
        
    e = None
    fg = None
    eg = None

============================================= engines ==============================================
 tag                                  : engines
 type                                 : Perkins 4008 TRS 1
 shaft_rating                         : 447 kW
 shaft_efficiency                     : 37.9 pct
 alternator_efficiency                : 94.8 pct
 dry_oxygen                           : 9 %w/w
 exhaust_temperature                  : 487 °C
 exhaust_back_pressure                : 5.9 kPag
 xa                                   : 0.15
 ng_afr                               : 11.45 mf
 sg_afr                               : 329.4 %mol/mol
 nu                                   : 0.02406 m³/mol
 ng_cv                                : 34.91 MJ/m³
 sg_cv                                : 11.79 MJ/m³
 derate                               : 0.9789
 shaft_rating_w_derate                : 437.6 kW
 thermal_input_rating                 : 1,179 kW
 rated_flow                           : 360.1 m³/h


## SACTO case 1 

In [ ]:
S[603].m = S[406].m - S[601].m - S[602].m 
with units["KN-601"] as c:
    c.design_massrate = S[603].m * 1.1
    c.gas = S[603]
    c.air = S[604]
    c.exhaust = S[607]

    ##################
    ##HIDDEN PORTION##
    ##################
    
    print(c)

====================================== KN-601 : SACTO Case 1 =======================================
 tag                                  : KN-601 : SACTO Case 1
 name                                 : SACTO Case 1
 design_massrate                      : 4.103 kg/h
 gas
    tag                               : 603 : Syngas to SACTO
    name                              : Syngas to SACTO
    fluid                             : syngas
    n                                 : 0.1811 kmol/h
    m                                 : 3.73 kg/h
    T                                 : 35 °C
    rho                               : 0.8144 kg/m³
    v                                 : 4.58 m³/h
    Q                                 : 0.2015 MJ/h
    P                                 : 111.3 kPa
    H_f                               : -22.29 MJ/h
    HHV                               : -56.34 MJ/h
    LHV                               : -51.37 MJ/h
 air
    tag                               : 604 : C

In [ ]:
energy = -S[602].HHV.to(ureg.MJ/ureg.s)
#eff = 0.36
    ##################
    ##HIDDEN PORTION##
    ##################
print(f"Waste water produced by GCU, Mass flow rate: {S[150].m}, Volume flow rate: {S[150].v}")

Electrical energy produced: 0.32 MJ/s
Assuming a 4000 kg/h gas recirculation rate, clean syngas after GCU: 231.44 m³/h
Amount of syngas available for export: 93.62 m³/h
Assuming CV of 43.4 MJ/kg of diesel for sustaining process: 37.77 kg/h
Total energy from cooling water: 197.08 MJ/h
Total energy from chilled water: 322.98 MJ/h
Waste water produced by GCU, Mass flow rate: 287.38 kg/h, Volume flow rate: 0.27 m³/h


In [ ]:
print(S[406])
print(S[602])

======================================== 406 : Clean syngas ========================================
 tag                                  : 406 : Clean syngas
 name                                 : Clean syngas
 fluid                                : syngas
 P                                    : 111.3 kPa
 T                                    : 35 °C
 n                                    : 9.153 kmol/h
 m                                    : 188.5 kg/h
 rho                                  : 0.8144 kg/m³
 v                                    : 231.4 m³/h
 Q                                    : 10.18 MJ/h
 H_f                                  : -1,126 MJ/h
 HHV                                  : -2,847 MJ/h
 LHV                                  : -2,596 MJ/h
 relative_humidity_operational        : 85.27 %mol/mol
-------------------------------- Composition Vectors / Coordinates ---------------------------------
                      N          M          Y          X
Symbol          

====================================== 602 : Syngas to Engine ======================================
 tag                                  : 602 : Syngas to Engine
 name                                 : Syngas to Engine
 fluid                                : syngas
 n                                    : 3.702 kmol/h
 m                                    : 76.24 kg/h
 T                                    : 35 °C
 rho                                  : 0.8144 kg/m³
 v                                    : 93.62 m³/h
 Q                                    : 4.12 MJ/h
 P                                    : 111.3 kPa
 H_f                                  : -455.6 MJ/h
 HHV                                  : -1,152 MJ/h
 LHV                                  : -1,050 MJ/h
-------------------------------- Composition Vectors / Coordinates ---------------------------------
                      N          M          Y          X
Symbol            mol/h       kg/h   %mol/mol       %w/w
───────